In [1]:
inputRDD = sc.textFile("/mnt/S3/data/dataSet10.txt")
inputRDD.first()

In [2]:
inputRDD.first()

In [3]:
filterRDD = inputRDD.filter(lambda line: len(line) > 0)
filterRDD.take(2)

In [4]:
import re

def removePunctuation(text):
    return re.sub("[\,\.\:\;\!\(\)\?]", "", text.lower().strip())

kvPair = filterRDD.map(lambda line: line.split(' ', 1))
kvPair = kvPair.flatMap(lambda line: [line[0] + ', ' + removePunctuation(line[1])])

In [5]:
kvPair.take(5)

In [6]:
kvPair = kvPair.map(lambda line: line.strip().split(","))
kvPair.take(2)

In [7]:
kvPair = kvPair.map(lambda line: (line[0], line[1].strip().split(" ")))
kvPair = kvPair.map(lambda lst: (lst[0], filter(None, lst[1])))
kvPair.take(2)

In [8]:
 def key_value_pairs(line):
    new_list = []
    for i in line[1]:
      new_list.append(line[0] + "~" + i)
    
    return new_list
  
splitList = kvPair.flatMap(lambda line: key_value_pairs(line))
splitList = splitList.map(lambda line: line.split("~"))
splitList.take(2)

In [9]:
invertedList = splitList.map(lambda x: (x[1], x[0]))
data = invertedList.distinct()  # .sortBy(lambda x: (x[1].split(':')[0], x[1].split(':')[1], x[1].split(':')[2]))
data.collect()

In [10]:
output = data.groupBy(lambda word: word[0]).sortByKey(ascending=True)
outputRDD = output.map(lambda word: (word[0], [i[1] for i in list(word[1])]))

In [11]:
def sortVerseIdList(verseIdList):
  newList = []
  for verseId in verseIdList:
    book, chapter, verse = verseId.split(":")
    newList += [(book, int(chapter), int(verse))]
  newList.sort()
  verseIdList = []
  for book, chapter, verse in newList:
    verseId = book + ':' + str(chapter) + ':' + str(verse)
    verseIdList += [ verseId ]
  return verseIdList

outputRDD = outputRDD.map(lambda x: (x[0], sortVerseIdList(x[1])))

In [12]:
outputRDD.take(5)

In [13]:
#outputRDD.coalesce(1).saveAsTextFile("/FileStore/output2-4")